In [ ]:
pip install -r ./requirements.txt

In [4]:
import os
import pandas as pd
import shutil

In [ ]:
%cd ../py

from constants import (
    CSV_EXTRACTED_PATH,
    CSV_MERGED_PATH,
    DATA_PATH,
    DATA_PRE_ANALYZED,
    TYPES,
)
from file.file_data_write import file_data_write
from main import main

%cd ../notebook

In [6]:
shutil.rmtree(CSV_EXTRACTED_PATH)
shutil.rmtree(CSV_MERGED_PATH)
shutil.rmtree(DATA_PATH)

In [ ]:
!bash ../scripts/unzip.sh

In [8]:
os.mkdir(CSV_MERGED_PATH)

In [9]:
for type in TYPES.keys():
    df = pd.DataFrame()

    for file in os.listdir(CSV_EXTRACTED_PATH):
        if file.endswith(f"{type}.csv"):
            file_df = pd.read_csv(
                f"{CSV_EXTRACTED_PATH}/{file}",
                encoding="latin1",
                engine="python",
            )

            file_df = file_df[TYPES[type].values()]

            if df.empty:
                df = file_df
            else:
                df = pd.concat([df, file_df])

    df.to_csv(
        f"{CSV_MERGED_PATH}/{type}.csv",
        encoding="latin1",
        index=False,
        sep=",",
    )

In [10]:
os.mkdir(DATA_PATH)

In [ ]:
for data_pre in DATA_PRE_ANALYZED.keys():
    file_data_write(main(*DATA_PRE_ANALYZED[data_pre]), data_pre)